## Multi-class Classification

In [347]:
## Import all the things
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from scipy.io import loadmat
import scipy.optimize as opt
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression

## Load data + general parameters

In [348]:
f = scipy.io.loadmat('ex3data1.mat')

# m by n ie 5000 samples by 400 features NUMPY ARRAY
X = f['X']

# m dim NUMPY ARRAY
y = f['y']

    
## Grab m (# number of samples) and n (# of features)
m, n = X.shape
    
# X_with_one_column.shape (5000, 401)
first_column = np.matrix('1 '*m).T
X_with_one_column = np.append(first_column, X, 1)

# X.shape, y.shape
learning_rate = 1

# theta is n+1 dimensional (+1 for element at index 0 which has no X)
theta = np.matrix('.01 '* (n+1))

## Write out functions previously

In [349]:
def sigmoid(z):
    return (1 / (1 + np.exp(-z)))

In [350]:
def cost2(theta, X, y, learning_rate):
    ######## SETUP #######
    ## Add column of 1s to X ... goes from being m by n to m by n+1
    ## (5000, 400) ......>>>>>> (5000, 401)
    first_column = np.matrix('1 '*y.size).T
    X_with_one_column = np.append(first_column, X, 1)
    
    ## Grab m (# number of samples) and n (# of features)
    m, n = X.shape
    
    ## Remove first theta when using regularization
    theta_without_first_term = np.delete(theta, 0)
    #####################
    
    # Should now be m-dimensional (1, 5000)
    X_theta = theta*X_with_one_column.T 
    
    # Should now be m-dimensional (1, 5000)
    h = sigmoid(X_theta) 
    
    # Scalar
    y_is_one = np.log(h)*(-y)
    y_is_zero =np.log(1-h)*(1 - y)
    cost = np.asscalar((y_is_one - y_is_zero) / m)
    regularization_term = (learning_rate * np.sum(np.power(theta_without_first_term, 2)))/(2*m)
    return cost + regularization_term

cost2(theta, X, y, learning_rate)

112.7918144159156

In [351]:
def cost(theta, X, y, learning_rate):
    y = y.T
    m = y.size
    theta_without_first_term = np.delete(theta, 0)
    
    # Should now be m-dimensional (1, 5000)
    X_theta = theta*X.T 
    
    # Should now be m-dimensional (1, 5000)
    h = sigmoid(X_theta) 
    
    # Scalar
    y_is_one = np.log(h)*(-y)
    y_is_zero =np.log(1-h)*(1 - y)
    cost = np.asscalar((y_is_one - y_is_zero) / m)
    regularization_term = (learning_rate * np.sum(np.power(theta_without_first_term, 2)))/(2*m)
    return cost + regularization_term

cost(theta, X_with_one_column, y.T, learning_rate)

112.7918144159156

In [352]:
def gradient2(theta, X, y, learning_rate):
    ################ SETUP ###############
    ## Add column of 1s to X ... goes from being m by n to m by n+1
    ## (5000, 400) ......>>>>>> (5000, 401)
    first_column = np.matrix('1 '*y.size).T
    X_with_one_column = np.append(first_column, X, 1)
    ## Grab m (# number of samples) and n (# of features)
    m, n = X.shape
    #####################################
    h = sigmoid(theta*X_with_one_column.T)
    diff = h - y.T
    first_term = (diff * X_with_one_column) / m
    reqularization_term = theta*(learning_rate/m)
    reqularization_term[0,0] = theta[0, 0]
    return reqularization_term + first_term

gradient2(theta, X, y, learning_rate);

In [353]:
def gradient(theta, X, y, learning_rate):
    y = y.T
    #m = y.size
    #h = sigmoid(theta*X.T)
    #diff = h - y.T
    #first_term = (diff * X) / m
    #reqularization_term = theta*(learning_rate/m)
    #reqularization_term[0,0] = theta[0, 0]
    #return reqularization_term + first_term
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learning_rate / len(X)) * theta)
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

gradient(theta, X_with_one_column, y.T, learning_rate)

array([-4.87015768e+00,  0.00000000e+00,  0.00000000e+00, -6.59186418e-08,
        3.03375475e-06,  1.85651595e-05, -6.90794664e-04, -8.80456507e-04,
       -3.67077352e-04, -1.10827298e-04, -1.32149427e-04, -3.93899051e-05,
        2.95906684e-05,  7.26919096e-05,  1.60579127e-04,  1.79557439e-04,
        1.04246826e-04,  3.22844010e-05,  2.27185250e-07, -3.43517453e-07,
        0.00000000e+00, -2.63613707e-07,  2.62168396e-06,  1.23763377e-06,
        4.88432398e-05,  8.59251343e-05, -1.11883866e-03, -3.98041022e-03,
       -5.55860517e-03, -4.30392409e-03, -1.21016777e-03,  1.04056335e-03,
        1.63769540e-03,  1.77891638e-03,  1.34886454e-03,  9.74691973e-04,
        6.15803555e-04,  2.79766072e-04,  1.61375792e-04,  4.44252054e-05,
        1.62656919e-05,  2.26168352e-06, -2.54673503e-05,  5.73484834e-05,
       -2.43904013e-04, -2.04429304e-03, -1.07639157e-02, -2.35359268e-02,
       -3.05914084e-02, -2.30009537e-02, -1.02480591e-02, -4.43011224e-03,
       -2.04719503e-03, -

In [356]:
# def one_vs_all(X, y, num_labels, learning_rate):
fmins = []
for i in range(1, 11):
    theta = np.zeros(n + 1)
    y_i = np.array([1 if label == i else 0 for label in y])
    y_i = np.reshape(y_i, (m, 1))
    fmins.append(minimize(fun=cost, x0=theta, args=(X_with_one_column, y_i.T, learning_rate), method='TNC', jac=gradient))
    #all_theta[i-1,:] = fmin.x
        
    #print y_i.shape, theta.shape

In [392]:
clfs = []
for i in range(1, 11):
    y_i = np.array([1 if label == i else 0 for label in y])
    lr = LogisticRegression()
    fit = lr.fit(X, y_i)
    clfs.append(fit)

In [393]:
for lr in clfs:
    meow = np.matrix(X[0])*lr.coef_.T
    lr.coef_.shape, lr.intercept_.shape, np.matrix(X[0]).shape
    print sigmoid(np.asscalar(meow) + np.asscalar(lr.intercept_))

2.633404544327858e-09
0.0006103940043565629
0.00017014038245174673
1.8589488407551303e-07
0.0015321872635531229
4.553540446662352e-05
6.476909426029784e-05
8.527510169337372e-05
0.0020051484952165917
0.9966491889741965
